<a href="https://colab.research.google.com/github/PinkPigmyPuff/SeatingChart/blob/main/ClassroomGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [261]:
import random
import numpy as np
from typing import List, Optional, Callable, Tuple


Chromosome = List[int]
Population = List[Chromosome]
Opinion = List[int]

In [262]:
def genChromosome(size: int) -> Chromosome:
    return random.sample(range(0, size), size)

def genPopulation(popSize: int, chromeosomeSize: int) -> Population:
    return [genChromosome(chromeosomeSize) for _ in range(popSize)]

def genOpinions(chromosome: Chromosome, opinionMax: int) -> Opinion:
  likes = []
  dislikes = []
  # for every student, randomly generate how many students they have opionions on, and add an empty array to like/dislike
  for i in range(0, len(chromosome)):
    likes.append([])
    dislikes.append([])
    likeNum = random.randint(0, opinionMax)
    dislikeNum = random.randint(0, opinionMax)
    dupelist = chromosome.copy()
    dupelist.pop(i)
    
    # choose random students that the current student likes
    for j in range(0, likeNum):
      randomStudent = random.randrange(0, len(dupelist))
      likes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

    # choose random students that the current student dislikes
    for j in range(0, dislikeNum):
      randomStudent = random.randrange(0, len(dupelist))
      dislikes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

  return likes, dislikes

In [263]:
def fitness(chromosome: Chromosome, peoplePerTable: int = 4, nextToFriendCost: int = 2, nextToEnemyCost: int = -1) -> int:
    size = len(chromosome)
    if size != len(likes) != len(dislikes):
        raise ValueError("genome and likes/dislikes must be of the same length")

    if size % peoplePerTable != 0:
        raise ValueError("The length of the chromosome must be divisable by peoplePerTable")
    # determine the cost of the given chromosome
    peopleHappiness = [0] * size
    chunkedClassroom = chunks(chromosome, peoplePerTable)
    # for every student
    for student in range(0, len(chromosome)):
      currentStudent = chromosome[student]
      studentsTable = 0
      # finds which table the currentStudent is sitting at
      if any(currentStudent in (match := nested_list) for nested_list in chunkedClassroom):
        studentsTable = chunkedClassroom.index(match)
        # print(f"current student is {currentStudent}, and they sit at table {studentsTable}")

      # determine the happiness of the currentStudent, based on each person at their table
      for tableMate in range(0, peoplePerTable):
        currentTableMate = chunkedClassroom[studentsTable][tableMate]
        if currentTableMate in likes[student]:
          peopleHappiness[student] += nextToFriendCost
        elif currentTableMate in dislikes[student]:
          peopleHappiness[student] += nextToEnemyCost

    return average(peopleHappiness)

def populationFitness(population: Population) -> float:
    return [fitness(chromosome) for chromosome in population]
    
# Return n-sized chunks from lst
def chunks(lst, n):
    x = [lst[i:i + n] for i in range(0, len(lst), n)]
    return(x)

def relu(x):
	  return max(0.0, x)
 
def average(lst):
   return float(sum(lst) / len(lst))

In [264]:
# Randomly chooses n individuals from the population and returns the fittest one
def tournamentSelection(population: Population, tournamentSize: int) -> Population:
		
		tournament = []
		for i in range(0, tournamentSize):
			random_id = random.randint(0, len(population)-1)
			tournament.append(population[random_id])


		#print(f"tournament pool! {tournament}")
		fittest = chooseHighest(tournament, 1)
		#print(f"fittest! {fittest}")
		return fittest

In [265]:
def roulette_wheel_selection(population: Population, populationFitness) -> Population:
    if len(population) != len(populationFitness):
        raise ValueError("genome and likes/dislikes must be of the same length")
    print(f"pFit: {populationFitness}")

    # Computes the totallity of the population fitness
    sumFitness = sum(populationFitness)
    print(f"sumFittness: {sumFitness}")
    
    # Computes for each chromosome the probability 
    chromosome_probabilities = [classroomHappiness/sumFitness for classroomHappiness in populationFitness]
    print(f"c_prob {chromosome_probabilities}")

    # Selects one chromosome based on the computed probabilities
    choice = population[np.random.choice(range(0, len(population)), p=chromosome_probabilities)]
    print(f"chioce_fitness {fitness(choice)}")
    return choice

In [266]:
def partiallyMatchedCrossover(ind1, ind2):
    #print(f"ind1: {ind1}\nind2: {ind2}")
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    #print(f"FINSIHED, ind1: {ind1}\n ind2: {ind2}")
    return ind1, ind2

In [267]:
def mutate(chromosome: Chromosome) -> Chromosome:
    length = len(chromosome)
    if length < 2:
      print("Chromosome is too short to be mutated")
      return chromosome
    index1, index2 = random.sample(range(0, length), 2)
    chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
    return chromosome

In [268]:
def chooseHighest(population: Population, num: int = 3):
  fit = populationFitness(population)
  highestIndex = 0
  for i in range(0, len(fit)):
      if fit[i] > fit[highestIndex]:
          highest = fit[i]
          highestIndex = i
  return population[highestIndex]

In [269]:
pop = genPopulation(12, 12)
likes, dislikes = genOpinions(pop[0], 4)

def run_evolution(
        generation_limit: int = 50,
        population = genPopulation(8, 8), 
        #likes = genOpinions(population[0], 4),
        #dislikes = genOpinions(population[0], 4)
        #peoplePerTable = 4,
        #nextToFriendCost = 1,
        #nextToEnemyCost = -1,
        ):
    random.shuffle(population)
    print(f"OG population: {population}")
    print(f"Likes: {likes}\nDislikes: {dislikes}")
    if len(population) % 2 != 0:
        print("Population size is odd, removing one chromosome")
        population.pop(0)

    for i in range(generation_limit):
        parents = [tournamentSelection(population, 5) for i in range(len(population))]
        #popFitness = populationFitness(population)
        #print(f"Generation {i} fitness: {popFitness}")
        print(f"population: {population}")
        #parents = [roulette_wheel_selection(population, popFitness) for i in range(len(population))]
        print(f"parents: {parents}")

        next_generation = []

        while parents:
            offspring1, offspring2 = partiallyMatchedCrossover(parents[0], parents[1])
            #print(f"nextPopulation CROSSED: {offspring1, offspring2}")
            next_generation.append(mutate(offspring1))
            next_generation.append(mutate(offspring2))
            #print(f"nextPopulation MUTATED: {next_generation}")
            parents = parents[2:]
            
        
        population = next_generation

    return chooseHighest(population)

run_evolution(population=pop)

OG population: [[10, 6, 3, 9, 2, 7, 11, 5, 0, 8, 4, 1], [4, 1, 7, 8, 6, 2, 5, 9, 0, 11, 10, 3], [7, 9, 10, 0, 1, 5, 4, 11, 2, 8, 6, 3], [7, 6, 1, 11, 10, 5, 0, 2, 8, 9, 4, 3], [2, 1, 9, 10, 5, 4, 11, 8, 0, 7, 6, 3], [9, 2, 1, 10, 7, 4, 0, 11, 5, 8, 6, 3], [8, 11, 6, 10, 3, 7, 0, 9, 5, 4, 2, 1], [0, 9, 7, 6, 1, 11, 2, 5, 4, 10, 8, 3], [7, 5, 11, 6, 8, 3, 1, 9, 2, 0, 4, 10], [9, 10, 11, 1, 8, 2, 7, 6, 3, 5, 4, 0], [9, 2, 5, 11, 3, 10, 4, 1, 7, 6, 0, 8], [7, 6, 9, 3, 8, 4, 10, 1, 0, 5, 11, 2]]
Likes: [[9], [], [0], [], [], [], [8, 4, 3], [], [1, 9, 0], [11, 5, 7], [], [5, 11, 2, 1]]
Dislikes: [[], [], [9], [0, 4, 11, 1], [7, 0], [10], [6, 1, 7], [1], [2, 7, 6], [1], [], [10]]
tournament pool! [[9, 2, 1, 10, 7, 4, 0, 11, 5, 8, 6, 3], [8, 11, 6, 10, 3, 7, 0, 9, 5, 4, 2, 1], [0, 9, 7, 6, 1, 11, 2, 5, 4, 10, 8, 3], [9, 2, 1, 10, 7, 4, 0, 11, 5, 8, 6, 3], [8, 11, 6, 10, 3, 7, 0, 9, 5, 4, 2, 1]]
fittest! [8, 11, 6, 10, 3, 7, 0, 9, 5, 4, 2, 1]
tournament pool! [[0, 9, 7, 6, 1, 11, 2, 5, 4, 10, 8

[4, 11, 2, 10, 7, 3, 1, 8, 6, 9, 5, 0]

1. Encoding scheme
- A chromosome is a permutation list
- Each index is an integer
- Each integer is a student in the class

2. Create the initial population
- First, a function to create a single chromsome
  - inputs chromosomeSize
  - outputs a chromosome of randomly shuffled integers

7. Testing
- A func